## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-07-47-23 +0000,bbc,"Get your act together, Post Office victims tel...",https://www.bbc.com/news/articles/crk6r2615vzo
1,2025-07-10-07-33-49 +0000,bbc,US cuts visa validity for most Nigerian applic...,https://www.bbc.com/news/articles/cx23ywg9w82o
2,2025-07-10-07-25-21 +0000,bbc,Royal Mail given go-ahead to scrap second-clas...,https://www.bbc.com/news/articles/c36x8k2612ko
3,2025-07-10-07-23-52 +0000,bbc,Deadly new Russian drone and missile attack hi...,https://www.bbc.com/news/articles/cj3rvpe06rxo
4,2025-07-10-07-02-53 +0000,nyt,U.S. Imposes Sanctions on U.N. Special Rapport...,https://www.nytimes.com/2025/07/10/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2298/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,16
23,new,6
8,cuts,5
164,israel,5
107,mexico,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
54,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,43
16,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,40
69,2025-07-09-12-26-31 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo,38
18,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo,32
22,2025-07-10-00-26-49 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,32


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
54,43,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
42,25,2025-07-09-19-48-23 +0000,nyt,Former President Yoon Suk Yeol of South Korea ...,https://www.nytimes.com/2025/07/09/world/asia/...
69,22,2025-07-09-12-26-31 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo
21,18,2025-07-10-00-36-34 +0000,bbc,"US sanctions UN expert Francesca Albanese, cri...",https://www.bbc.com/news/articles/c70rllxr0kyo
73,18,2025-07-09-10-55-24 +0000,nyt,Jailed Kurdish Leader Declares the End to PKK’...,https://www.nytimes.com/2025/07/09/world/middl...
23,15,2025-07-09-23-26-54 +0000,nyt,Mexico Sentences 10 Men to 140 Years Each in P...,https://www.nytimes.com/2025/07/09/world/ameri...
0,14,2025-07-10-07-47-23 +0000,bbc,"Get your act together, Post Office victims tel...",https://www.bbc.com/news/articles/crk6r2615vzo
7,14,2025-07-10-05-35-00 +0000,wsj,June’s U.K. Home Buyer Demand Turns Positive f...,https://www.wsj.com/economy/central-banking/ju...
31,14,2025-07-09-21-34-15 +0000,nyt,Yemen’s Houthi Militia Took Sailors Hostage Af...,https://www.nytimes.com/2025/07/09/world/middl...
6,12,2025-07-10-05-44-06 +0000,bbc,'You need to shape up': Trump's pick for Singa...,https://www.bbc.com/news/articles/cgrxd7d78r0o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
